# 神經網路入門
本章，我們會將上一章節所學的知識應用於三個新問題，此問題涵蓋最常見的三個使用場景：(1).二分類問題、(2).多分類問題、(3).標量回歸問題。

### 3.1 神經網路剖析
前面幾張介紹過，訓練神經網路主要圍繞以下四個面向：
* 「層」、多少層組合成網路( 或模型 )。
* 「輸入數據」、相對應的「目標」。
* 「損失函數」，即用於學習的反饋訊號。
* 「優化器」，決定學習過程如何進行。<br><br>

我們可以將四者關係可視化，如圖3.1所示，我們可以藉由多個層鏈接在一起，組成網路，將輸入數據映射為預測值。然後損失函數將這些預測值與相對應的目標進行比較後，得到損失值，其可用於衡量網路預測值與預期結果的匹配程度。再用優化器使用這個損失值來更新網路的權重。<br>
![3.1，網路、層、損失函數、優化器](../img/3.1.PNG)<br>
接下來，我們來進一步研究層、網路、損失函數、優化器之間的關係。


### 3.1.1 層：深度學習的基礎組件
層是一個數據處理模塊，將輸入張量轉換為輸出張量。我們可以將層想成DL的樂高積木，Keras等框架則將這種比喻具體化，建構DL模型就是將「相互兼容」的多個層拼接在一起，以建立有用的數據變換流程。這裡，「層兼容性( layer compatibility )」具體指得是每一層只接受特定形狀的輸入張量，並返回特定形狀的輸出張量。舉例如下。


In [30]:
'''
我們要創立一個層，只接受輸入維度大小為784的2D張量，
這個層將返回一個維度大小為32的張量。
'''
from keras import layers

layers = layers.Dense(32, input_shape=(784, ))      # 有32個輸出位元的全連接層

###
# 因此，這個層後面只能接受32維度向量作為輸入的層。
# 但別擔心，keras會自動匹配輸入層的形狀，如下：
###

from keras import models
from keras import layers

models = models.Sequential()
models.add(layers.Dense(32, input_shape=(784, )))
models.add(layers.Dense(32))

###
# 其中，第二層沒有指定輸入數據形狀的參數( input_shape )，
# 相反，Keras可以自動推導出輸入形狀等於上一層的輸出形狀!!!
###


### 3.1.2 模型：層構成的網路
DL模型是由層構成的有向無環圖，亦即任意頂點出發無法經過若干條邊回到該點。最常見的例子就是層的線性堆疊，將單一輸入映射為單一輸出。<br>
當然，還有更多類型的網路結構，舉一些常見的網路拓樸結構如下：<br>
* 雙分支(　two-branch )網路
* 多頭( multihead )網路
* Inception 模塊 <br><br>

「網路拓樸」結構定義了一個假設空間( hypothesis space )又稱為可能性空間。回想一下第1章節ML的定義："在預先定義好的可能性問題中，利用反饋訊號來尋找輸入數據的有用表示。"<br><br>

一旦選定了網路拓樸結構，意味著將可能性空間(假設空間)限定為一系列特定的張量運算，將輸入數據映射為輸出數據。然後，我們還需要對這些張量運算的權重張量找到一組合適的值。


### 3.1.3 損失函數與優化器：配置學習過程的關鍵
一旦確定了網路架構，我們還需要選擇以下兩個參數。
* 損失函數( 目標函數 )：在訓練過程中需要將其最小化，他能夠衡量當前任務是否已成功完成。
* 優化器：決定如何基於損失函數對網路進行更新。<br><br>

開始實現keras前，我們考慮幾個重要的課題：
* 當NN具有多個損失函數時：需所有Loss_function取平均，變成單一指標值。
* 如何選正確的目標函數：想想我們建立一個愚蠢又無所不不能的AI，讓全體人類的平均幸福感最大化會發生甚麼事情?
* 所面對的研究問題是甚麼：(1).二分類問題、(2).多分類問題、(3).標量回歸問題。每個問題可對應不同的損失函數，如，(1).->二元交叉熵損失函數、(2).分類交叉熵損失函數、(3).回歸問題 -> MSE損失函數 、 序列問題 -> 連結主義時序分類(CTC)損失函數。


## 3.2 Keras 簡介
我們已經見過一個Keras模型的示例，就是MNIST的例子。典型的Keras工作流程就和那個例子類似。
* (1).定義訓練數據：輸入張量和目標張量。
* (2).定義層組成的網路( 或模型 )，將輸入映射到目標。
* (3).配置學習過程：選擇損失函數、優化器和需要監控的指標。
* (4).調用模型的fit方法在訓練數據上進行迭代。<br>
定義模型有兩種方法：一種是利用Sequential類，另一種是函數式API( functional API ，用於層組成的有無循環圖，讓我們可以構建任意形式的架構)。我們將兩者實現如下：


In [47]:
###
# 首先是利用Sequential類定義的兩層模型
# (注意，我們向第一層傳入了輸入數據的預期形狀)
###
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(784,)))
model.add(layers.Dense(10, activation='softmax'))

###
# 下面使用函數式API定義的相同模型
# 利用函數式API，我們可以操縱模型的數據張量
# 並且將層應用於這個張量
# 就好像這些層式函數一樣
###

input_tensor = layers.Input(shape=(784,)) 
x = layers.Dense(32, activation='relu')(input_tensor)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=input_tensor, outputs=output_tensor)


一旦定義好了模型架構，接下來配置學習過程是在「編譯」這一步，我們需要指定模型使用的(1).優化器、(2).損失函數以及(3).學習過程所關心的「監控指標」，如下：<br><br>

from keras import optimizers    <br><br>

model.compile(optimizer=optimizers.RMSprop(lr=0.001), <br>
              loss='mse',                      
              metrics=['accuracy']) <br><br>
              
最後，學習過程就是通過fit () 方法將輸入數據的Numpy模組( 和對應的目標數據 )傳入模型，如下：<br>

model.fit(input_tensor, target_tensor, batch_size=128, epochs=10)<br>


# 本章小結
* 本章，我們學習了：二分類問題、多分類問題以及標量回歸問題。
* 在輸入數據之前，我們應該先對原始輸入數據進行預處理。
* 如果數據特徵具有不同的取值範圍，那麼需對數據進行特徵縮放。
* 隨著訓練進行，NN最終會過擬合，並在前所未見的數據上得到很差的結果。
* 如果訓練數據不是很多，應該使用小型的網路結構，以避免過擬合問題發生。
* 如果數據被分為多個類別，那麼中間層過小可能會導致訊息瓶頸。
* 回歸問題使用的損失函數與評估指標都與分類問題不同。
* 如果要處理的數據很少，K折驗證有助於評估模型。